<a href="https://colab.research.google.com/github/jserrataylor/kike/blob/main/Ejercicio_para_Kike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Instalcion de Dependencias

## requirements.txt

streamlit
scikit-learn
numpy
joblib

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import joblib

# 1. Generar datos sintéticos
np.random.seed(42)
n_samples = 200
data = pd.DataFrame({
    'age': np.random.randint(18, 90, size=n_samples),
    'income': np.random.uniform(20000, 100000, size=n_samples),
    'num_visits': np.random.randint(0, 50, size=n_samples),
})

# Utilizamos una función logística para definir la probabilidad de compra
# Coeficientes arbitrarios para simulación
coef = np.array([0.03, 0.00002, 0.05])
intercept = -5
X = data.values
logit = X.dot(coef) + intercept
prob = 1 / (1 + np.exp(-logit))
data['purchased'] = (np.random.rand(n_samples) < prob).astype(int)

# 2. División entrenamiento / prueba
X = data[['age', 'income', 'num_visits']]
y = data['purchased']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Definición y entrenamiento de modelos
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    results[name] = acc
    print(f"{name}: Accuracy = {acc:.4f}")

# 4. Selección del mejor modelo
best_name = max(results, key=results.get)
best_model = models[best_name]
print(f"\nMejor modelo: {best_name} con accuracy = {results[best_name]:.4f}")

# 5. Guardar el mejor modelo
joblib.dump(best_model, 'best_model.pkl')
print("Modelo guardado como 'best_model.pkl'")


LogisticRegression: Accuracy = 0.7000
RandomForest: Accuracy = 0.6250
GradientBoosting: Accuracy = 0.6500

Mejor modelo: LogisticRegression con accuracy = 0.7000
Modelo guardado como 'best_model.pkl'


In [ ]:
## SCRIPT DESPLIEGUE MODELO SUPERVISADO CLASIFICACION

import streamlit as st
import joblib
import numpy as np

# Título y descripción
st.title("Predicción de Compra de Cliente")
st.write("""
Este servicio despliega un modelo de clasificación entrenado sobre datos sintéticos
y predice la probabilidad de que un cliente compre un producto en función de:
- Edad
- Ingresos anuales
- Número de visitas al sitio
""")

# Función para cargar y cachear el modelo como recurso
@st.cache_resource
def load_model():
    return joblib.load("best_model.pkl")

model = load_model()

# Panel lateral para los parámetros de entrada
st.sidebar.header("Parámetros de entrada")
age = st.sidebar.slider("Edad", min_value=18, max_value=90, value=30, step=1)
income = st.sidebar.number_input("Ingresos anuales (USD)", min_value=0.0, value=50000.0, step=1000.0)
visits = st.sidebar.slider("Número de visitas al sitio", min_value=0, max_value=50, value=5, step=1)

# Preparar los datos de entrada y hacer la predicción
X_new = np.array([[age, income, visits]])
prob = model.predict_proba(X_new)[0, 1]
pred = model.predict(X_new)[0]

# Mostrar resultados
st.subheader("Resultados de la predicción")
st.markdown(f"- **Probabilidad de compra:** {prob:.2%}")
st.markdown(f"- **Predicción:** {'🛒 Comprar' if pred == 1 else '🚫 No comprar'}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import joblib

# 1. Generar datos sintéticos
np.random.seed(42)
n_samples = 500
data = pd.DataFrame({
    'sqft': np.random.uniform(500, 4000, n_samples),
    'rooms': np.random.randint(1, 6, n_samples),
    'age': np.random.uniform(0, 100, n_samples),
    'dist': np.random.uniform(0, 30, n_samples)  # distancia a la ciudad en km
})

# Fórmula sintética para precio (USD)
noise = np.random.normal(0, 20000, n_samples)
data['price'] = (
    50 * data['sqft'] +
    10000 * data['rooms'] -
    300 * data['age'] -
    500 * data['dist'] +
    noise
)

# 2. División entrenamiento / prueba
X = data[['sqft', 'rooms', 'age', 'dist']]
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Definición y entrenamiento de modelos de regresión
models = {
    'LinearRegression': LinearRegression(),
    'RandomForestRegressor': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoostingRegressor': GradientBoostingRegressor(n_estimators=100, random_state=42),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    r2 = r2_score(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    results[name] = {'r2': r2, 'mse': mse}
    print(f"{name}: R2 = {r2:.4f}, MSE = {mse:.2f}")

# 4. Selección del mejor modelo (por R2)
best_name = max(results, key=lambda k: results[k]['r2'])
best_model = models[best_name]
print(f"\nMejor modelo: {best_name} con R2 = {results[best_name]['r2']:.4f}")

# 5. Guardar el mejor modelo
joblib.dump(best_model, 'best_model_regresion.pkl')
print("Modelo guardado como 'best_model_regresion.pkl'")


LinearRegression: R2 = 0.8471, MSE = 426071265.32
RandomForestRegressor: R2 = 0.8223, MSE = 495281497.34
GradientBoostingRegressor: R2 = 0.8163, MSE = 511982087.71

Mejor modelo: LinearRegression con R2 = 0.8471
Modelo guardado como 'best_model_regresion.pkl'


In [ ]:
## SCRIPT DESPLIEGUE MODELO SUPERVISADO DE REGRESION

import streamlit as st
import joblib
import numpy as np
import os

# Título y descripción
st.title("Predicción de Precio de Venta de Casas")
st.write("""
Esta aplicación carga un modelo de regresión entrenado sobre datos sintéticos
y predice el precio de venta de una casa según:
- Superficie en pies cuadrados (`sqft`)
- Número de habitaciones (`rooms`)
- Antigüedad de la vivienda en años (`age`)
- Distancia al centro de la ciudad en km (`dist`)
""")

# 1. Localizar la carpeta donde está este script
CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))

# 2. Ruta al modelo guardado en la misma carpeta
MODEL_PATH = os.path.join(CURRENT_DIR, "best_model_regresion.pkl")

# 3. Función para cargar y cachear el modelo como recurso
@st.cache_resource
def load_model(path):
    if not os.path.exists(path):
        st.error(f"❌ No se encontró el modelo en:\n`{path}`\nAsegúrate de haber subido `best_model.pkl`.")
        return None
    return joblib.load(path)

model = load_model(MODEL_PATH)
if model is None:
    st.stop()

# 4. Panel lateral: parámetros de entrada
st.sidebar.header("Parámetros de Entrada")
sqft  = st.sidebar.slider("Superficie (sqft)",      min_value=100,  max_value=5000,  value=1500, step=50)
rooms = st.sidebar.slider("Habitaciones",           min_value=1,    max_value=10,    value=3,    step=1)
age   = st.sidebar.slider("Antigüedad (años)",      min_value=0,    max_value=150,   value=20,   step=1)
dist  = st.sidebar.slider("Distancia al centro (km)", min_value=0.0,  max_value=100.0, value=10.0, step=0.5)

# 5. Preparar input y predecir
X_new      = np.array([[sqft, rooms, age, dist]])
pred_price = model.predict(X_new)[0]

# 6. Mostrar resultado
st.subheader("🔍 Resultado de la Predicción")
st.write(f"💰 **Precio estimado:** USD {pred_price:,.2f}")
